In [80]:
%reload_ext autoreload
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from pipetools import pipe, utils, where

In [2]:
team_urls = {
    "liverpool": "https://fbref.com/en/squads/822bd0ba/Liverpool-Stats",
    "aston_villa": "https://fbref.com/en/squads/8602292d/Aston-Villa-Stats",
    "leeds_united": "https://fbref.com/en/squads/5bfb9659/Leeds-United-Stats",
    "crystal_palace": "https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats",
    "chelsea": "https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats",
    "leicester_city": "https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats",
    "wolves": "https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats",
    "tottenham_hotspurs": "https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats",
    "westham_united": "https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats",
    "manchester_city": "https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats",
    "everton": "https://fbref.com/en/squads/d3fd31cc/Everton-Stats",
    "southampton": "https://fbref.com/en/squads/33c895d4/Southampton-Stats",
    "newcastle_united": "https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats",
    "manchester_united": "https://fbref.com/en/squads/19538871/Manchester-United-Stats",
    "brighton": "https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats",
    "westbrom": "https://fbref.com/en/squads/60c6b05f/West-Bromwich-Albion-Stats",
    "burnley": "https://fbref.com/en/squads/943e8050/Burnley-Stats",
    "sheffield_united": "https://fbref.com/en/squads/1df6b87e/Sheffield-United-Stats",
    "fulham": "https://fbref.com/en/squads/fd962109/Fulham-Stats",
    "arsenal": "https://fbref.com/en/squads/18bb7c10/Arsenal-Stats"
}

In [3]:
len(team_urls)

20

### Brewing the soup

In [4]:
liv_content = requests.get(team_urls["arsenal"])

In [5]:
liv_content.status_code

200

In [6]:
liv_soup = BeautifulSoup(liv_content.content)

### Shooting Stats

In [7]:
shooting = BeautifulSoup(liv_content.content).find(attrs={"id": "all_stats_shooting_10728"})

In [8]:
len(BeautifulSoup(shooting.contents[5]).find_all("tr")[2:-1])

19

In [9]:
shooting = BeautifulSoup(shooting.contents[5]).find_all("tr")[2:-1]

In [11]:
import collections

ShootingStats = collections.namedtuple(
    'ShootingStats', 
    'name profile_url nationality position age minutes_90s goals shots_total shots_on_target \
    shots_on_target_pct shots_total_per90 goals_per_shot goals_per_shot_on_target \
    shots_free_kicks pens_made pens_att xg npxg npxg_per_shot xg_net npxg_net')

In [100]:
def extract_shooting_stats(shooting: list) -> list(ShootingStats):
    shooting_stats = []
    for stat in shooting:
        shooting_stats.append(ShootingStats(
            name=(stat.find("th").text),
            profile_url=f'https://fbref.com{stat.find("a").attrs["href"]}',
            nationality=stat.find(attrs={"data-stat": "nationality"}).text.split(" "),
            position=stat.find(attrs={"data-stat": "position"}).text,
            age=format_age(stat.find(attrs={"data-stat": "age"}).text),
            minutes_90s=float(stat.find(attrs={"data-stat": "minutes_90s"}).text),
            goals=int(stat.find(attrs={"data-stat": "goals"}).text),
            shots_total=int(stat.find(attrs={"data-stat": "shots_total"}).text),
            shots_on_target=int(stat.find(attrs={"data-stat": "shots_on_target"}).text),
            shots_on_target_pct=to_float(stat.find(attrs={"data-stat": "shots_on_target_pct"}).text),
            shots_total_per90=to_float(stat.find(attrs={"data-stat": "shots_total_per90"}).text),
            goals_per_shot=to_float(stat.find(attrs={"data-stat": "goals_per_shot"}).text),
            goals_per_shot_on_target=to_float(stat.find(attrs={"data-stat": "goals_per_shot_on_target"}).text),
            shots_free_kicks=to_float(stat.find(attrs={"data-stat": "shots_free_kicks"}).text),
            pens_made=int(stat.find(attrs={"data-stat": "pens_made"}).text),
            pens_att=int(stat.find(attrs={"data-stat": "pens_att"}).text),
            xg=to_float(stat.find(attrs={"data-stat": "xg"}).text),
            npxg=to_float(stat.find(attrs={"data-stat": "npxg"}).text),
            npxg_per_shot=to_float(stat.find(attrs={"data-stat": "npxg_per_shot"}).text),
            xg_net=stat.find(attrs={"data-stat": "xg_net"}).text,
            npxg_net=stat.find(attrs={"data-stat": "npxg_net"}).text
        ))

In [101]:
shooting_stats[-1]

ShootingStats(name='Shkodran Mustafi', profile_url='https://fbref.com/en/players/3f2d59fe/Shkodran-Mustafi', nationality=['de', 'GER'], position='DF', age='28 years and 192 days', minutes_90s=0.5, goals=0, shots_total=0, shots_on_target=0, shots_on_target_pct=0.0, shots_total_per90=0.0, goals_per_shot=0.0, goals_per_shot_on_target=0.0, shots_free_kicks=0.0, pens_made=0, pens_att=0, xg=0.0, npxg=0.0, npxg_per_shot=0.0, xg_net='0.0', npxg_net='0.0')

### Passing Stats

In [102]:
passing = BeautifulSoup(liv_content.content).find(attrs={"id": "all_stats_passing_10728"})

In [103]:
len(BeautifulSoup(passing.contents[5]).find_all("tr")[2:-1])

19

In [104]:
passing = BeautifulSoup(passing.contents[5]).find_all("tr")[2:-1]

In [105]:
PassingStats = collections.namedtuple(
    'PassingStats',
    'name passes_completed passes passes_pct passes_total_distance passes_progressive_distance \
    passes_completed_short passes_short passes_pct_short passes_completed_medium passes_medium \
    passes_pct_medium passes_completed_long passes_long passes_pct_long assists xa xa_net assisted_shots \
    passes_into_final_third passes_into_penalty_area crosses_into_penalty_area progressive_passes'
)

In [110]:
def extract_passing_stats(passing: list) -> list(PassingStats):
    passing_stats = []
    for stat in passing:
        passing_stats.append(
            PassingStats(
                name=stat.find("th").text,
                passes_completed=int(stat.find(attrs={"data-stat": "passes_completed"}).text),
                passes=int(stat.find(attrs={"data-stat": "passes"}).text),
                passes_pct=to_float(stat.find(attrs={"data-stat": "passes_pct"}).text),
                passes_total_distance=to_float(stat.find(attrs={"data-stat": "passes_total_distance"}).text),
                passes_progressive_distance=to_float(stat.find(attrs={"data-stat": "passes_progressive_distance"}).text),
                passes_completed_short=int(stat.find(attrs={"data-stat": "passes_completed_short"}).text),
                passes_short=int(stat.find(attrs={"data-stat": "passes_short"}).text),
                passes_pct_short=to_float(stat.find(attrs={"data-stat": "passes_pct_short"}).text),
                passes_completed_medium=int(stat.find(attrs={"data-stat": "passes_completed_medium"}).text),
                passes_medium=int(stat.find(attrs={"data-stat": "passes_medium"}).text),
                passes_pct_medium=to_float(stat.find(attrs={"data-stat": "passes_pct_medium"}).text),
                passes_completed_long=int(stat.find(attrs={"data-stat": "passes_completed_long"}).text),
                passes_long=int(stat.find(attrs={"data-stat": "passes_long"}).text),
                passes_pct_long=to_float(stat.find(attrs={"data-stat": "passes_pct_long"}).text),
                assists=int(stat.find(attrs={"data-stat": "assists"}).text),
                xa=to_float(stat.find(attrs={"data-stat": "xa"}).text),
                xa_net=stat.find(attrs={"data-stat": "xa_net"}).text,
                assisted_shots=int(stat.find(attrs={"data-stat": "assisted_shots"}).text),
                passes_into_final_third=int(stat.find(attrs={"data-stat": "passes_into_final_third"}).text),
                passes_into_penalty_area=int(stat.find(attrs={"data-stat": "passes_into_penalty_area"}).text),
                crosses_into_penalty_area=int(stat.find(attrs={"data-stat": "crosses_into_penalty_area"}).text),
                progressive_passes=int(stat.find(attrs={"data-stat": "progressive_passes"}).text)
            )
        )

In [111]:
passing_stats[0]

PassingStats(name='Pierre-Emerick Aubameyang', passes_completed=116, passes=162, passes_pct=71.6, passes_total_distance=1865.0, passes_progressive_distance=404.0, passes_completed_short=0, passes_short=7, passes_pct_short=0.0, passes_completed_medium=101, passes_medium=121, passes_pct_medium=83.5, passes_completed_long=15, passes_long=34, passes_pct_long=44.1, assists=1, xa=1.1, xa_net='-0.1', assisted_shots=8, passes_into_final_third=5, passes_into_penalty_area=6, crosses_into_penalty_area=3, progressive_passes=20)

### Extra Passing stats

In [112]:
extra_passing = BeautifulSoup(liv_content.content).find(attrs={"id": "all_stats_passing_types_10728"})

In [114]:
len(BeautifulSoup(extra_passing.contents[5]).find_all("tr")[2:-1])

19

In [115]:
extra_passing = BeautifulSoup(extra_passing.contents[5]).find_all("tr")[2:-1]

In [116]:
ExtraPassingStats = collections.namedtuple(
    'ExtraPassingStats',
    'name passes_live passes_dead passes_free_kicks through_balls passes_pressure passes_switches\
    crosses corner_kicks corner_kicks_in corner_kicks_out corner_kicks_straight passes_ground\
    passes_low passes_high passes_left_foot passes_right_foot passes_head throw_ins passes_other_body\
    passes_completed passes_offsides passes_oob passes_intercepted passes_blocked'
)

In [124]:
def extract_extra_passing_stats(extra_passing: list) -> list(ExtraPassingStats):
    extra_passing_stats = []
    for stat in extra_passing:
        extra_passing_stats.append(
            ExtraPassingStats(
                name=stat.find("th").text,
                passes_live=int(stat.find(attrs={"data-stat": "passes_live"}).text),
                passes_dead=int(stat.find(attrs={"data-stat": "passes_dead"}).text),
                passes_free_kicks=int(stat.find(attrs={"data-stat": "passes_free_kicks"}).text),
                through_balls=int(stat.find(attrs={"data-stat": "through_balls"}).text),
                passes_pressure=int(stat.find(attrs={"data-stat": "passes_pressure"}).text),
                passes_switches=int(stat.find(attrs={"data-stat": "passes_switches"}).text),
                crosses=int(stat.find(attrs={"data-stat": "crosses"}).text),
                corner_kicks=int(stat.find(attrs={"data-stat": "corner_kicks"}).text),
                corner_kicks_in=int(stat.find(attrs={"data-stat": "corner_kicks_in"}).text),
                corner_kicks_out=int(stat.find(attrs={"data-stat": "corner_kicks_out"}).text),
                corner_kicks_straight=int(stat.find(attrs={"data-stat": "corner_kicks_straight"}).text),
                passes_ground=int(stat.find(attrs={"data-stat": "passes_ground"}).text),
                passes_low=int(stat.find(attrs={"data-stat": "passes_low"}).text),
                passes_high=int(stat.find(attrs={"data-stat": "passes_high"}).text),
                passes_left_foot=int(stat.find(attrs={"data-stat": "passes_left_foot"}).text),
                passes_right_foot=int(stat.find(attrs={"data-stat": "passes_right_foot"}).text),
                passes_head=int(stat.find(attrs={"data-stat": "passes_head"}).text),
                throw_ins=int(stat.find(attrs={"data-stat": "throw_ins"}).text),
                passes_other_body=int(stat.find(attrs={"data-stat": "passes_other_body"}).text),
                passes_completed=int(stat.find(attrs={"data-stat": "passes_completed"}).text),
                passes_offsides=int(stat.find(attrs={"data-stat": "passes_offsides"}).text),
                passes_oob=int(stat.find(attrs={"data-stat": "passes_oob"}).text),
                passes_intercepted=int(stat.find(attrs={"data-stat": "passes_intercepted"}).text),
                passes_blocked=int(stat.find(attrs={"data-stat": "passes_blocked"}).text)
            )
        )

In [125]:
extra_passing_stats[0]

ExtraPassingStats(name='Pierre-Emerick Aubameyang', passes_live=160, passes_dead=2, passes_free_kicks=0, through_balls=4, passes_pressure=36, passes_switches=2, crosses=16, corner_kicks=0, corner_kicks_in=0, corner_kicks_out=0, corner_kicks_straight=0, passes_ground=104, passes_low=30, passes_high=28, passes_left_foot=17, passes_right_foot=130, passes_head=6, throw_ins=0, passes_other_body=3, passes_completed=116, passes_offsides=2, passes_oob=3, passes_intercepted=3, passes_blocked=5)

### Goal Creation Stats

In [126]:
gca = BeautifulSoup(liv_content.content).find(attrs={"id": "all_stats_gca_10728"})

In [127]:
len(BeautifulSoup(gca.contents[5]).find_all("tr")[2:-1])

19

In [128]:
gca = BeautifulSoup(gca.contents[5]).find_all("tr")[2:-1]

In [129]:
GCAStats = collections.namedtuple(
    'GCAStats',
    'name sca sca_per90 sca_passes_live sca_passes_dead sca_dribbles sca_shots sca_fouled gca\
    gca_per90 gca_passes_live gca_passes_dead gca_dribbles gca_shots gca_fouled gca_og_for'
)

In [135]:
def extract_gca_stats(gca: list) -> list(GCAStats):
    gca_stats = []
    for stat in gca:
        gca_stats.append(
            GCAStats(
                name=stat.find("th").text,
                sca=int(stat.find(attrs={"data-stat": "sca"}).text),
                sca_per90=to_float(stat.find(attrs={"data-stat": "sca_per90"}).text),
                sca_passes_live=int(stat.find(attrs={"data-stat": "sca_passes_live"}).text),
                sca_passes_dead=int(stat.find(attrs={"data-stat": "sca_passes_dead"}).text),
                sca_dribbles=int(stat.find(attrs={"data-stat": "sca_dribbles"}).text),
                sca_shots=int(stat.find(attrs={"data-stat": "sca_shots"}).text),
                sca_fouled=int(stat.find(attrs={"data-stat": "sca_fouled"}).text),
                gca=int(stat.find(attrs={"data-stat": "gca"}).text),
                gca_per90=to_float(stat.find(attrs={"data-stat": "gca_per90"}).text),
                gca_passes_live=int(stat.find(attrs={"data-stat": "gca_passes_live"}).text),
                gca_passes_dead=int(stat.find(attrs={"data-stat": "gca_passes_dead"}).text),
                gca_dribbles=int(stat.find(attrs={"data-stat": "gca_dribbles"}).text),
                gca_shots=int(stat.find(attrs={"data-stat": "gca_shots"}).text),
                gca_fouled=int(stat.find(attrs={"data-stat": "gca_fouled"}).text),
                gca_og_for=int(stat.find(attrs={"data-stat": "gca_og_for"}).text)
            )
        )

In [136]:
gca_stats[0]

GCAStats(name='Pierre-Emerick Aubameyang', sca=16, sca_per90=2.67, sca_passes_live=12, sca_passes_dead=0, sca_dribbles=1, sca_shots=1, sca_fouled=2, gca=2, gca_per90=0.33, gca_passes_live=2, gca_passes_dead=0, gca_dribbles=0, gca_shots=0, gca_fouled=0, gca_og_for=0)

### Defensive Stats

In [137]:
defensive = BeautifulSoup(liv_content.content).find(attrs={"id": "all_stats_defense_10728"})

In [138]:
len(BeautifulSoup(defensive.contents[5]).find_all("tr")[2:-1])

19

In [139]:
defensive = BeautifulSoup(defensive.contents[5]).find_all("tr")[2:-1]

In [140]:
DefensiveActions = collections.namedtuple(
    'DefensiveActions',
    'name tackles tackles_won tackles_def_3rd tackles_mid_3rd tackles_att_3rd dribble_tackles dribbles_vs\
    dribble_tackles_pct dribbled_past pressures pressure_regains pressure_regain_pct pressures_def_3rd\
    pressures_mid_3rd pressures_att_3rd blocks blocked_shots blocked_shots_saves blocked_passes interceptions\
    tackles_interceptions clearances errors'
)

In [141]:
def extract_defensive_actions(defensive_actions: list) -> list(DefensiveActions):
    defensive_actions = []
    for stat in defensive:
        defensive_actions.append(
            DefensiveActions(
                name=stat.find("th").text,
                tackles=int(stat.find(attrs={"data-stat": "tackles"}).text),
                tackles_won=int(stat.find(attrs={"data-stat": "tackles_won"}).text),
                tackles_def_3rd=int(stat.find(attrs={"data-stat": "tackles_def_3rd"}).text),
                tackles_mid_3rd=int(stat.find(attrs={"data-stat": "tackles_mid_3rd"}).text),
                tackles_att_3rd=int(stat.find(attrs={"data-stat": "tackles_att_3rd"}).text),
                dribble_tackles=int(stat.find(attrs={"data-stat": "dribble_tackles"}).text),
                dribbles_vs=int(stat.find(attrs={"data-stat": "dribbles_vs"}).text),
                dribble_tackles_pct=to_float(stat.find(attrs={"data-stat": "dribble_tackles_pct"}).text),
                dribbled_past=int(stat.find(attrs={"data-stat": "dribbled_past"}).text),
                pressures=int(stat.find(attrs={"data-stat": "pressures"}).text),
                pressure_regains=int(stat.find(attrs={"data-stat": "pressure_regains"}).text),
                pressure_regain_pct=to_float(stat.find(attrs={"data-stat": "pressure_regain_pct"}).text),
                pressures_def_3rd=int(stat.find(attrs={"data-stat": "pressures_def_3rd"}).text),
                pressures_mid_3rd=int(stat.find(attrs={"data-stat": "pressures_mid_3rd"}).text),
                pressures_att_3rd=int(stat.find(attrs={"data-stat": "pressures_att_3rd"}).text),
                blocks=int(stat.find(attrs={"data-stat": "blocks"}).text),
                blocked_shots=int(stat.find(attrs={"data-stat": "blocked_shots"}).text),
                blocked_shots_saves=int(stat.find(attrs={"data-stat": "blocked_shots_saves"}).text),
                blocked_passes=int(stat.find(attrs={"data-stat": "blocked_passes"}).text),
                interceptions=int(stat.find(attrs={"data-stat": "interceptions"}).text),
                tackles_interceptions=int(stat.find(attrs={"data-stat": "tackles_interceptions"}).text),
                clearances=int(stat.find(attrs={"data-stat": "clearances"}).text),
                errors=int(stat.find(attrs={"data-stat": "errors"}).text),
            )
        )

In [142]:
defensive_actions[0]

DefensiveActions(name='Pierre-Emerick Aubameyang', tackles=5, tackles_won=2, tackles_def_3rd=0, tackles_mid_3rd=4, tackles_att_3rd=1, dribble_tackles=2, dribbles_vs=5, dribble_tackles_pct=40.0, dribbled_past=3, pressures=84, pressure_regains=20, pressure_regain_pct=23.8, pressures_def_3rd=15, pressures_mid_3rd=37, pressures_att_3rd=32, blocks=6, blocked_shots=0, blocked_shots_saves=0, blocked_passes=6, interceptions=0, tackles_interceptions=5, clearances=3, errors=0)

### Possession Stats

In [144]:
possesion = BeautifulSoup(liv_content.content).find(attrs={"id": "all_stats_possession_10728"})

In [145]:
len(BeautifulSoup(possesion.contents[5]).find_all("tr")[2:-1])

19

In [150]:
possesion = BeautifulSoup(possesion.contents[5]).find_all("tr")[2:-1]

In [151]:
PossessionStats = collections.namedtuple(
    'PossessionStats',
    'name touches touches_def_pen_area touches_def_3rd touches_mid_3rd touches_att_3rd touches_att_pen_area\
    touches_live_ball dribbles_completed dribbles dribbles_completed_pct players_dribbled_past nutmegs\
    carries carry_distance carry_progressive_distance pass_targets passes_received passes_received_pct\
    miscontrols dispossessed'
)

In [153]:
def extract_possession_stats(possesion: list) -> list(PossessionStats):
    possesion_stats = []
    for stat in possesion:
        possesion_stats.append(
            PossessionStats(
                name=stat.find("th").text,
                touches=int(stat.find(attrs={"data-stat": "touches"}).text),
                touches_def_pen_area=int(stat.find(attrs={"data-stat": "touches_def_pen_area"}).text),
                touches_def_3rd=int(stat.find(attrs={"data-stat": "touches_def_3rd"}).text),
                touches_mid_3rd=int(stat.find(attrs={"data-stat": "touches_mid_3rd"}).text),
                touches_att_3rd=int(stat.find(attrs={"data-stat": "touches_att_3rd"}).text),
                touches_att_pen_area=int(stat.find(attrs={"data-stat": "touches_att_pen_area"}).text),
                touches_live_ball=int(stat.find(attrs={"data-stat": "touches_live_ball"}).text),
                dribbles_completed=int(stat.find(attrs={"data-stat": "dribbles_completed"}).text),
                dribbles=int(stat.find(attrs={"data-stat": "dribbles"}).text),
                dribbles_completed_pct=to_float(stat.find(attrs={"data-stat": "dribbles_completed_pct"}).text),
                players_dribbled_past=int(stat.find(attrs={"data-stat": "players_dribbled_past"}).text),
                nutmegs=int(stat.find(attrs={"data-stat": "nutmegs"}).text),
                carries=int(stat.find(attrs={"data-stat": "carries"}).text),
                carry_distance=int(stat.find(attrs={"data-stat": "carry_distance"}).text),
                carry_progressive_distance=int(stat.find(attrs={"data-stat": "carry_progressive_distance"}).text),
                pass_targets=int(stat.find(attrs={"data-stat": "pass_targets"}).text),
                passes_received=int(stat.find(attrs={"data-stat": "passes_received"}).text),
                passes_received_pct=to_float(stat.find(attrs={"data-stat": "passes_received_pct"}).text),
                miscontrols=int(stat.find(attrs={"data-stat": "miscontrols"}).text),
                dispossessed=int(stat.find(attrs={"data-stat": "dispossessed"}).text),
            )
        )

In [154]:
possesion_stats[0]

PossessionStats(name='Pierre-Emerick Aubameyang', touches=204, touches_def_pen_area=3, touches_def_3rd=26, touches_mid_3rd=85, touches_att_3rd=106, touches_att_pen_area=18, touches_live_ball=202, dribbles_completed=3, dribbles=8, dribbles_completed_pct=37.5, players_dribbled_past=4, nutmegs=0, carries=130, carry_distance=921, carry_progressive_distance=529, pass_targets=270, passes_received=165, passes_received_pct=61.1, miscontrols=5, dispossessed=6)

### Playing Time Stats

In [156]:
playing_time = BeautifulSoup(liv_content.content).find(attrs={"id": "all_stats_playing_time_10728"})

In [157]:
len(BeautifulSoup(playing_time.contents[5]).find_all("tr")[2:-1])

22

In [158]:
playing_time = BeautifulSoup(playing_time.contents[5]).find_all("tr")[2:-1]

In [159]:
PlayingTimeStats = collections.namedtuple(
    'PlayingTimeStats',
    'name games minutes minutes_per_game minutes_pct minutes_90s games_starts minutes_per_start\
    games_subs minutes_per_sub unused_subs points_per_match on_goals_for on_goals_against plus_minus\
    plus_minus_per90 plus_minus_wowy on_xg_for on_xg_against xg_plus_minus xg_plus_minus_per90\
    xg_plus_minus_wowy'
)

In [167]:
def extract_playing_time_stats(playing_time: list) -> list(PlayingTimeStats):
    playing_time_stats = []
    for stat in playing_time:
        playing_time_stats.append(
            PlayingTimeStats(
                name=stat.find("th").text,
                games=int(stat.find(attrs={"data-stat": "games"}).text),
                minutes=to_float(stat.find(attrs={"data-stat": "minutes"}).text),
                minutes_per_game=to_float(stat.find(attrs={"data-stat": "minutes_per_game"}).text),
                minutes_pct=to_float(stat.find(attrs={"data-stat": "minutes_pct"}).text),
                minutes_90s=to_float(stat.find(attrs={"data-stat": "minutes_90s"}).text),
                games_starts=int(stat.find(attrs={"data-stat": "games_starts"}).text),
                minutes_per_start=to_float(stat.find(attrs={"data-stat": "minutes_per_start"}).text),
                games_subs=int(stat.find(attrs={"data-stat": "games_subs"}).text),
                minutes_per_sub=to_float(stat.find(attrs={"data-stat": "minutes_per_sub"}).text),
                unused_subs=int(stat.find(attrs={"data-stat": "unused_subs"}).text),
                points_per_match=to_float(stat.find(attrs={"data-stat": "points_per_match"}).text),
                on_goals_for=to_float(stat.find(attrs={"data-stat": "on_goals_for"}).text),
                on_goals_against=to_float(stat.find(attrs={"data-stat": "on_goals_against"}).text),
                plus_minus=stat.find(attrs={"data-stat": "plus_minus"}).text,
                plus_minus_per90=stat.find(attrs={"data-stat": "plus_minus_per90"}).text,
                plus_minus_wowy=stat.find(attrs={"data-stat": "plus_minus_wowy"}).text,
                on_xg_for=to_float(stat.find(attrs={"data-stat": "on_xg_for"}).text),
                on_xg_against=to_float(stat.find(attrs={"data-stat": "on_xg_against"}).text),
                xg_plus_minus=stat.find(attrs={"data-stat": "xg_plus_minus"}).text,
                xg_plus_minus_per90=stat.find(attrs={"data-stat": "xg_plus_minus_per90"}).text,
                xg_plus_minus_wowy=stat.find(attrs={"data-stat": "xg_plus_minus_wowy"}).text,
            )
        )

In [168]:
playing_time_stats[0]

PlayingTimeStats(name='Pierre-Emerick Aubameyang', games=6, minutes=540.0, minutes_per_game=90.0, minutes_pct=100.0, minutes_90s=6.0, games_starts=6, minutes_per_start=90.0, games_subs=0, minutes_per_sub=0.0, unused_subs=0, points_per_match=1.5, on_goals_for=8.0, on_goals_against=7.0, plus_minus='+1', plus_minus_per90='+0.17', plus_minus_wowy='', on_xg_for=6.5, on_xg_against=7.9, xg_plus_minus='-1.3', xg_plus_minus_per90='-0.22', xg_plus_minus_wowy='')

### Misc Stats

In [169]:
misc = BeautifulSoup(liv_content.content).find(attrs={"id": "all_stats_misc_10728"})

In [170]:
len(BeautifulSoup(misc.contents[5]).find_all("tr")[2:-1])

19

In [171]:
misc = BeautifulSoup(misc.contents[5]).find_all("tr")[2:-1]

In [172]:
MiscStats = collections.namedtuple(
    'MiscStats',
    'name cards_yellow cards_red cards_yellow_red fouls fouled offsides crosses interceptions\
    tackles_won pens_won pens_conceded own_goals ball_recoveries aerials_won aerials_lost aerials_won_pct'
)

In [173]:
def extract_misc_stats(misc: list) -> list(MiscStats):
    misc_stats = []
    for stat in misc:
        misc_stats.append(
            MiscStats(
                name=stat.find("th").text,
                cards_yellow=int(stat.find(attrs={"data-stat": "cards_yellow"}).text),
                cards_red=int(stat.find(attrs={"data-stat": "cards_red"}).text),
                cards_yellow_red=int(stat.find(attrs={"data-stat": "cards_yellow_red"}).text),
                fouls=int(stat.find(attrs={"data-stat": "fouls"}).text),
                fouled=int(stat.find(attrs={"data-stat": "fouled"}).text),
                offsides=int(stat.find(attrs={"data-stat": "offsides"}).text),
                crosses=int(stat.find(attrs={"data-stat": "crosses"}).text),
                interceptions=int(stat.find(attrs={"data-stat": "interceptions"}).text),
                tackles_won=int(stat.find(attrs={"data-stat": "tackles_won"}).text),
                pens_won=int(stat.find(attrs={"data-stat": "pens_won"}).text),
                pens_conceded=int(stat.find(attrs={"data-stat": "pens_conceded"}).text),
                own_goals=int(stat.find(attrs={"data-stat": "own_goals"}).text),
                ball_recoveries=int(stat.find(attrs={"data-stat": "ball_recoveries"}).text),
                aerials_won=int(stat.find(attrs={"data-stat": "aerials_won"}).text),
                aerials_lost=int(stat.find(attrs={"data-stat": "aerials_lost"}).text),
                aerials_won_pct=to_float(stat.find(attrs={"data-stat": "aerials_won_pct"}).text),
            )
        )

In [174]:
misc_stats[0]

MiscStats(name='Pierre-Emerick Aubameyang', cards_yellow=1, cards_red=0, cards_yellow_red=0, fouls=4, fouled=5, offsides=6, crosses=16, interceptions=0, tackles_won=2, pens_won=0, pens_conceded=0, own_goals=0, ball_recoveries=21, aerials_won=6, aerials_lost=9, aerials_won_pct=40.0)

### Utilities

In [37]:
def format_age(age: str) -> str:
    spl = age.split("-")
    return f"{spl[0]} years and {spl[1]} days"

In [98]:
def to_float(text: str) -> float:
    if not text:
        return 0.0
    else:
        return text > pipe | str | float

In [99]:
type(to_float('0.222'))

float

## FPL Data